In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [0]:
%cd '/content/drive/My Drive/shared-works/Thread-1'

/content/drive/My Drive/shared-works/Thread-1


In [0]:
!pip install ftfy

In [0]:
from utils_clf import reform_dataset_for_model1 as reform
import time

ModuleNotFoundError: ignored

In [0]:
st = time.time()

In [0]:
!ls './Data/auxiliary/'

clf_ids0.pkl   labels7.pkl	  preds3_masks.pkl     tokens14_masks.pkl
clf_ids10.pkl  labels8.pkl	  preds4_masks.pkl     tokens15_masks.pkl
clf_ids11.pkl  labels9.pkl	  preds5_masks.pkl     tokens1_masks.pkl
clf_ids12.pkl  metadata0.pkl	  preds6_masks.pkl     tokens2_masks.pkl
clf_ids13.pkl  metadata10.pkl	  preds7_masks.pkl     tokens3_masks.pkl
clf_ids14.pkl  metadata11.pkl	  preds8_masks.pkl     tokens4_masks.pkl
clf_ids15.pkl  metadata12.pkl	  preds9_masks.pkl     tokens5_masks.pkl
clf_ids1.pkl   metadata13.pkl	  tfrecord	       tokens6_masks.pkl
clf_ids2.pkl   metadata14.pkl	  tfrecord0.tfrecord   tokens7_masks.pkl
clf_ids3.pkl   metadata15.pkl	  tfrecord10.tfrecord  tokens8_masks.pkl
clf_ids4.pkl   metadata1.pkl	  tfrecord11.tfrecord  tokens9_masks.pkl
clf_ids5.pkl   metadata2.pkl	  tfrecord12.tfrecord  triple0_pars_list.pkl
clf_ids6.pkl   metadata3.pkl	  tfrecord13.tfrecord  triple10_pars_list.pkl
clf_ids7.pkl   metadata4.pkl	  tfrecord14.tfrecord  triple11_pars_list.pkl
clf_ids8.

In [0]:
st = time.time()
reform()
print(time.time() - st)

Reloading ...


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from init_model import Transformer as PreProcessor
from gen_model import Transformer as Generator
from text_utils import TextEncoder#, encode_text
from utils import iter_data, Logger
import time
import json
import numpy as np
import tensorflow.keras as keras

In [0]:
def encode_text(texts, n_ctx=512, n_special = 1, n_vocab=40478, encoder=None):
    if encoder == None:
        ENCODER_PATH = 'model/encoder_bpe_40000.json'
        BPE_PATH = 'model/vocab_40000.bpe'
        encoder = TextEncoder(ENCODER_PATH, BPE_PATH)

    tokens = encoder(texts, verbose=False)
    n_batch = len(tokens)

    inputs = np.zeros((1, len(tokens[0][:n_ctx-1]) + len(tokens[1][:n_ctx-1]) + 2 + n_ctx, 3), dtype=np.int32)

    seq_len = 0
    for i, x in enumerate(tokens):
        x1 = x[:n_ctx-1]
        inputs[0, seq_len : seq_len+len(x1), 0] = x1
        inputs[0, seq_len+len(x1), 0] = n_vocab
        inputs[0, seq_len : seq_len+len(x1)+1, 1] = np.arange(n_vocab + n_special, n_vocab + n_special + len(x1)+1)
        inputs[0, seq_len : seq_len+len(x1)+1, 2] = n_vocab + n_special + n_ctx + 2*i
        seq_len += len(x1) + 1

    inputs[0, seq_len : , 1] = np.arange(n_vocab + n_special, n_vocab + n_special + n_ctx)
    inputs[0, seq_len : , 2] = n_vocab + n_special + n_ctx + 1

    return seq_len, inputs

In [0]:
ENCODER_PATH = 'model/encoder_bpe_40000.json'
BPE_PATH = 'model/vocab_40000.bpe'
encoder = TextEncoder(ENCODER_PATH, BPE_PATH)
decoder = encoder.decoder

In [0]:
n_vocab = 40478
n_ctx = 512
n_special = 1
n_segment = 3

In [0]:
sess =  tf.Session()
keras.backend.set_session(sess)
preprocessor = PreProcessor("preprocessor", n_vocab)
generator = Generator("generator", n_vocab)

In [0]:
X = tf.placeholder(tf.int32, [1, None, 3])
L = tf.placeholder(tf.int32, [])
token, mem_k, mem_v = preprocessor([L, X])

W0705 12:19:59.612984 140345441539968 deprecation_wrapper.py:119] From /content/drive/My Drive/shared-works/Thread-1/init_model.py:55: The name tf.keras.initializers.random_normal is deprecated. Please use tf.compat.v1.keras.initializers.random_normal instead.

W0705 12:19:59.614973 140345441539968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0705 12:19:59.983514 140345441539968 deprecation_wrapper.py:119] From /content/drive/My Drive/shared-works/Thread-1/init_model.py:120: The name tf.matrix_band_part is deprecated. Please use tf.linalg.band_part instead.

W0705 12:20:00.068338 140345441539968 deprecation_wrapper.py:119] From /content/drive/My Drive/shared-works/Thread-1/i

In [0]:
T = tf.placeholder(tf.int32, [1, 1, 3])
K = tf.placeholder(tf.float32, [12, 1, 12, 64, None])
V = tf.placeholder(tf.float32, [12, 1, 12, None, 64])
I = tf.placeholder(tf.int32, [])
next_token, next_mem_k, next_mem_v = generator([T, K, V, I], k=25)

W0705 12:20:09.182770 140345441539968 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0705 12:20:09.193585 140345441539968 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:253: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


In [0]:
preprocessor.load_weights('./checkpoints/language_model/model/cp-0125.ckpt')
generator.load_weights('./checkpoints/language_model/model/cp-0125.ckpt')

In [0]:
txt = [
    "In the morning, Macduff finds Duncan dead, and Macbeth slaughters the sleeping servants to prevent their denial of the murder. Macduff and Lennox, a Scottish noble, discuss that as Malcolm has fled to England that puts him under suspicion of being involved in the murder. They also admire Macbeth's justice on the supposedly-treacherous servants. With Malcolm gone, Macbeth is asked to become King of Scotland."
    ,
    "During the evening, Macbeth makes a comment about Banquo not keeping his promise to be at the feast. When the assassins arrive, Macbeth asks if they have succeeded and is enraged when he finds out that Fleance has escaped. Then Macbeth sees Banquo's ghost in his armour standing among other guests next to the table. Macbeth is afraid and starts to talk to the ghost which is unseen by anyone else present. Lady Macbeth tries to calm everyone by claiming that her husband is unwell, but Macbeth continues to rave, which prompts Macduff and his wife to leave the feast despite the King ordering them to stay. Lady Macbeth dismisses all the guests and takes Macbeth back to their chamber."
]

In [0]:
txt = [
    "Chigurh, hired to recover the money, arrives to search Moss's home, where he uses his bolt pistol to blow the lock out of the door. Investigating the break in, Terrell County Sheriff Ed Tom Bell observes the blown-out lock. Following an electronic tracking device hidden in the money, Chigurh goes to Moss's motel room and kills a group of Mexicans who are waiting to ambush Moss. Moss has rented a second room adjacent to the Mexicans' room with access to the duct where the money is hidden. He retrieves the briefcase just before Chigurh opens the duct and finds it empty."
    ,
    "Moss retrieves the case from the bank of the Rio Grande and arranges to meet Carla Jean at a motel in El Paso, where he plans to give her the money and hide her from danger. Carla Jean is approached by Sheriff Bell, who promises to protect Moss. Carla Jean's mother unwittingly reveals Moss' location to a group of Mexicans who had been tailing them. Bell reaches the motel rendezvous at El Paso, only to hear gunshots and spot a pickup truck speeding from the motel. As Bell enters the parking lot, he sees Moss lying dead. When Carla Jean arrives, she chokes up upon discovering her husband is dead."
]

In [0]:
txt = [
    "Eli visits Daniel, who is hung over, in Daniel's private bowling alley. Eli, now a radio preacher, offers to sell Daniel drilling rights on the land of William Bandy, who has recently died."
    ,
    "Shaken, Eli confesses to being in dire financial straits and to having strayed morally. Daniel taunts him, chases him around the bowling alley, and shoots him to death with a gun."
]

# txt= [
#     'once there was a prince whom had not cut his yellow hair in decades. there was a saying in the town that he was under some magicians spell. People looked at his hair and thought unworthy thoughts.',
#     'the prince was seen riding into the sunset with his mighty horse, with his red hair cut as short as can be, happily singing the ancient songs of his ancestors.'
    
# ]

# txt = ['the commet was about to hit the spaceship. All of the spaceships sheilds had been activated. Everyone on board was termendously anxious and the alarms were hammering on the ears of the passengers. For a moment all the lights went out and the alarms went silent. there was a distant sound of a gaint vacum cleaner.',
#       'Jack woke up with a great pain in his back and chest. His head was pounding and his eyesight was dim. With great difficulty he managed to stand up, a thing being dizzy -as he was- made it so hard. After a moment or so he looked at his sroundings: it was disaster. Everything was destroyed.']


txt=[
    'suzan, an Irish girl sick of living in a small town in ireland, travels to America when she is twenty. In New York, suzan lives at a Brooklyn boarding house with other young women. She has a job at a department store but her shyness and sad demeanor with customers garners some criticism from Miss Fortini, her supervisor. suzan has difficulty adjusting to her new life and a letter from her sister makes her even more homesick. Father Flood gets her enrolled in bookkeeping classes as Eilis wants to eventually become an accountant. At an Irish dance, she meets Italian-American Tony Fiorello. They begin dating and she gradually grows more comfortable living in New York as their romance becomes more serious.'
    ,
    "Once back in Ireland at the age of twenty four, suzan falls into a new life, temporarily taking her late sister's old bookkeeping job, and being set up with a well-off bachelor. It's a completely different life than the one she left behind for Brooklyn. She doesn't open any of Tony's letters."
    
]

In [0]:
# import nltk
# nltk.download('punkt')

In [0]:
init_len, x = encode_text(txt)
init_len

210

In [0]:
gen_len = 400

In [0]:
t, k, v = sess.run([token, mem_k, mem_v], {X: x, L: init_len - 1})

In [0]:
# x[0, :, 2] += 1

In [0]:
x[0, init_len, 0] = t
gen_len -= 1

In [0]:
gen_tokens = [decoder[t].replace("</w>", "")]

In [0]:
for step in range(gen_len):
    prev_token = x[:, init_len + step : init_len + step + 1, :]
    t, k, v = sess.run([next_token, next_mem_k, next_mem_v], {T: prev_token, K: k, V: v, I: init_len + step})
    gen_tokens.append(decoder[t].replace("</w>", ""))
    x[0, init_len + step + 1, 0] = t

In [0]:
' '.join(gen_tokens)

"the three remaining girls of the group who have been married since the age of sixteen begin living apart . at the age of sixty , they are married to each other . they are living a charmed life . their wedding year is the same year , but they are both in their thirties and have had their first romance . the one they marry is in the first year of their married life . it is a beautiful , happy - looking , beautiful , sunny , sunny , sunny and happy - looking summer - year , and as they live they enjoy each other 's company and are able to make their lives happy again . they go back to new york and enjoy each other 's company . they go through their wedding years and enjoy their lives together . they have had no one to talk to them about the things they have done in the past . they have been a great deal happier , they have had no more accidents to keep them apart , the love they have had for each other is a good thing , but it is the first time since they had married that they have had s

In [0]:
with open('./Data/triple0_pars_list.pkl', 'rb') as pkl:
    pars = pickle.load(pkl)

In [0]:
len(pars)

449534

In [0]:
p = pars[list(pars)[2000]]

In [0]:
[pp[0] for pp in p]

[' "Mother, we\'re afloat agin." It was a gruff, sleepy voice that spoke, and the old fisherman turned over and snored on, as though the fact of their home being afloat was of no consequence to him. His wife, however, was by no means so easy in her mind, for it was only during the equinoctial gales and an unusually high tide that their home was lifted from its moorings; and now it had been swinging and swaying for hours, and the rusty chains that held it fast to some posts were creaking and straining as though the next gust of wind would certainly carry them out to sea or drive them up the river, where they would inevitably be swamped in a very short time, for their boat-home was leaky at the bottom--had been a water-logged boat before the fisherman took possession of it and turned it into a quaint-looking cottage by running up some wooden walls along the sides, and roofing it in with planks and tarpaulin. Thus converted into a dwelling-house, the boat had been secured, by four chains 

In [0]:
len(pars[40961][0])

3